In [1]:
import pandas as pd
import json 
import re
import math 
import ast 

from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm
import tiktoken

import os
from dotenv import load_dotenv

load_dotenv()
os.getenv('OPENAI_API_KEY')

client = AsyncOpenAI()
model = "gpt-4o-mini"

In [13]:
def read_parquet(uploaded_file):
    return pd.read_parquet(uploaded_file)

df = read_parquet("/Users/anne/Desktop/4_channels_predictions_09_2023_09_2024.parquet")
df.reset_index(inplace=True)
df_exploded = df.explode('claims', ignore_index=True)
df_normalized = pd.json_normalize(df_exploded['claims'])
df_result = df_exploded.drop(columns=['claims']).join(df_normalized)
df_claims = df_result[(df_result['disinformation_score']=='high') & (df_result['pro_anti']!='pro-écologie')]
df_claims

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker
67,f17cb36a314156417616ef1ebb9232bbf203a1ecdc983c...,2023-09-04 05:56:00,série qu'on peut redire par fonction juré déjà...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169379984302...",2,559,Cette allégation semble associée à une interpr...,le giec est transhumaniste,Dans un passage en parlant des personnages du ...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
87,e57e39657004072e5f998d19e6a8a952ee8f35923b9072...,2023-09-04 07:50:00,salle d'attente il y a toutes les classes rage...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""giec"", ""timestamp"": 169380664205...",2,554,Cette allégation semble utiliser le terme 'tra...,"La mention du GIEC comme un ""transhumaniste"".",Dans un extrait de programme où différents per...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
251,5b7207073227562ad5fd7b252e290d0a2079f3cbbb2e5c...,2023-09-06 07:22:00,en france on peut quand même s'appuyer sur ce ...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences_indirecte...","[{""keyword"": ""incendies"", ""timestamp"": 1693977...",2,594,Cette allégation semble hautement exagérée et ...,Il y a eu beaucoup de morts tués par des pharm...,Lors d'une discussion sur le rôle des pharmaci...,N/A,narrative,high,anti-écologie,j'ai l'impression qu'il y a eu beaucoup de mor...,narrative
259,53b6b045672a2327c58c72a38a3ddb603587ad43421f8c...,2023-09-06 08:38:00,je suis même allé plus loin je vais essayer de...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences"", ""ressou...","[{""keyword"": ""\u00e9coresponsable"", ""timestamp...",2,614,"Cette allégation frôle la théorie du complot, ...",Le réchauffement climatique est causé par une ...,Lors d'une discussion sur les enjeux environne...,N/A,consensus,high,anti-écologie,Le réchauffement climatique est causé par une ...,narrative
313,6c3bb9299d31332c644b54832399976117d28776c9e22c...,2023-09-07 08:56:00,pour faire réussir nos enfants dans quel monde...,europe1,True,Information - Magazine,L'heure des pros,"[""changement_climatique_constat""]","[{""keyword"": ""climatique"", ""timestamp"": 169406...",1,563,Si le changement climatique est parfois utilis...,Le changement climatique est traité comme une ...,Le commentaire sur une polémique entre personn...,N/A,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73775,f875741067fda8011d4aa9ad4c543e52227e31b525fed1...,2024-07-29 13:14:00,sur place voilà ce qu'ils restent par exemple ...,tf1,False,Information - Journal,JT 13h,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""feux"", ""timestamp"": 172225164506...",1,446,Cette allégation est sérieuse et nécessite une...,L'incendie serait d'origine volontaire.,Le journaliste fait état des enquêtes en cours...,N/A,consensus,high,anti-écologie,d'après les autorités le feu serait d'origine ...,narrative
74166,cdfed233896d49971897df448465dd43b79ae871b33adb...,2024-08-05 09:16:00,les insectes non on pense que c'est la chaleur...,tf1,False,Information - Magazine,Bonjour ! La Matinale,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""g\u00e9othermie"", ""timestamp"": 1...",2,479,Ce claim fait l'impasse sur le fait que la gén...,La climatisation industrialisée a été une répo...,"Il est mentionné que la climatisation, qui est...",N/A,consensus,high,anti-écologie,le progrès sur le réchauffement climatique la ...,

In [3]:
system_prompt = f"""
Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. 
Je vais te donner des claims sur des thématiques environnementales extraits de retranscriptions d'un flux TV ou Radio à factchecker. 
Pour chaque claim, donne une catégorisation de cette allégation suivant la classification qui suit.
Renvoie le résultat en json sans autre phrase d'introduction ou de conclusion avec à chaque fois le champ suivant : "cards_label_pred"

<catégories prédéfinies>
- 0_accepted: Lorsque l'ensemble de l'extrait est neutre ou reflète le consensus scientifique sur les questions climatiques, sans contenir de désinformation. Cette catégorie est aussi adaptée si l'extrait n'aborde pas le sujet du climat.
- 1_its_not_happening: Le réchauffement climatique n'a pas lieu. Cette catégorie soutient que le changement climatique est inexistant ou largement exagéré. Elle suggère que des facteurs naturels — tels que le froid, la préservation de la glace et le refroidissement des océans — prouvent que le réchauffement n'est pas en cours. De plus, elle implique que les prévisions de changements climatiques sévères, y compris la montée du niveau de la mer et les phénomènes météorologiques extrêmes, sont exagérées ou inventées. Certaines affirmations soutiennent que la terminologie climatique a été manipulée (par exemple, en passant de "réchauffement climatique" à "changement climatique") pour alarmer le public sans raison.
- 2_its_not_us: Les humains ne sont pas à l'origine du réchauffement climatique. Cette catégorie avance que les activités humaines, en particulier les émissions de gaz à effet de serre, ne sont pas responsables du réchauffement observé. Elle soutient que les cycles naturels, l'activité solaire ou d'autres facteurs non humains sont à l'origine des changements climatiques. Les affirmations de cette catégorie minimisent souvent le rôle du CO2, ou soulignent que les changements climatiques passés se sont produits sans intervention humaine, suggérant que le réchauffement actuel est également naturel.
- 3_its_not_bad: Les impacts climatiques ne sont pas mauvais. Cette catégorie reconnaît qu'un certain réchauffement pourrait se produire, mais affirme qu'il pose peu de risques. Les arguments de cette catégorie suggèrent que la sensibilité du climat au CO2 est faible, ce qui limiterait le réchauffement. Les partisans prétendent que l'environnement, les espèces et la santé humaine seront largement épargnés, car les plantes bénéficient de l'augmentation du CO2, et que de nombreuses espèces, comme les ours polaires, peuvent s'adapter. Dans l'ensemble, elle suggère que les impacts climatiques seront mineurs et gérables.
- 4_solutions_wont_work: Les solutions climatiques ne fonctionneront pas. Cette catégorie s'oppose aux politiques climatiques proposées, en affirmant qu'elles seront nuisibles économiquement ou socialement, inefficaces ou irréalistes. Elle soutient que les politiques de réduction des émissions augmenteront les coûts, affaibliront la sécurité nationale et restreindront les libertés individuelles sans bénéfices climatiques significatifs. D'autres affirmations proposent que les solutions "vertes" sont peu fiables, que l'énergie propre est non éprouvée, ou que les combustibles fossiles sont abondants et essentiels pour l'avenir. Essentiellement, elle soutient que les politiques climatiques sont trop coûteuses, difficiles ou inefficaces pour être poursuivies.
- 5_science_is_unreliable: Les mouvements pour le climat et la science du climat ne sont pas fiables. Cette catégorie attaque la crédibilité de la science climatique et du mouvement pour le climat, affirmant qu'ils sont non fiables, biaisés ou idéologiquement orientés. Elle remet en question le consensus scientifique sur le changement climatique, suggérant que les données, modèles et méthodes sont défectueux. De plus, elle présente le mouvement climatique comme alarmiste, biaisé ou faisant partie d'une conspiration plus large, motivée par des intentions politiques ou religieuses plutôt qu'une véritable préoccupation. Ce cadre cherche à miner la confiance dans la science climatique et dans ceux qui plaident pour l'action.
</catégories prédéfinies>
    """

encoding = tiktoken.encoding_for_model(model)
prompt_tokens = len(encoding.encode(system_prompt))

async def classify_claim(
        text: str, system_prompt: str = system_prompt, prompt_tokens: str = prompt_tokens, tpm_limit: int = 2e6
    ):
    text_tokens = len(encoding.encode(text))
    total_tokens = prompt_tokens + text_tokens
    wait_time_s = 60 * total_tokens / tpm_limit
    await asyncio.sleep(wait_time_s)
    
    response = await client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text},
        ],
        temperature=0,
    )
    return response.choices[0].message.content

async def run_classification(texts: list[str]):
    semaphore = asyncio.Semaphore(50)
    
    async def bounded_classify_claim(text):
        async with semaphore:
            return await classify_claim(text)
    
    classification = await tqdm.gather(*[bounded_classify_claim(text) for text in texts])
    return classification

In [ ]:
classification = await run_classification(df_claims["claim"])
cards_label_preds = []

for item in classification:
    # Remove ```json and ``` 
    cleaned_item = re.sub(r'```json\n|```', '', item)

    # Load as JSON dictionary
    try:
        parsed_item = json.loads(cleaned_item)
        cards_label_preds.append(parsed_item["cards_label_pred"])
    except json.JSONDecodeError:
        print(f"Parsing error: {cleaned_item}")

cards_label_preds
df_claims["cards_label_pred"] = cards_label_preds

df_exploded["cards_label_pred"] = ''
df_exploded.loc[df_claims.index, "cards_label_pred"] = df_claims["cards_label_pred"]

def add_key(row):
    try:
        return {**row['claims'], 'cards_label_pred': row['cards_label_pred']}
    except:
        return row['claims']

df_exploded['claims'] = df_exploded.apply(lambda row: add_key(row), axis=1)
df_exploded = df_exploded.drop(columns=['cards_label_pred'])
df_grouped = df_exploded.groupby('id', as_index=False).agg({'claims': list})

def isnan_(row):
    try:
        return math.isnan(row.claims[0])
    except:
        return False

df.loc[~ df_grouped.apply(lambda row: isnan_(row), axis=1), 'claims'] = df_grouped.loc[~ df_grouped.apply(lambda row: isnan_(row), axis=1), 'claims']
# df.to_csv("/Users/anne/Desktop/4_channels_predictions_09_2023_09_2024_with_cards.csv")

In [69]:
df_cards = pd.read_csv("/Users/anne/Desktop/4_channels_predictions_09_2023_09_2024_with_cards.csv")
df_cards.drop(columns={'Unnamed: 0'}, inplace=True)

df_cards['claims'] = df_cards['claims'].apply(lambda x: re.sub(r'}\s*{', '}, {', x))
df_cards['claims'] = df_cards['claims'].apply(lambda x: ast.literal_eval(x))

In [60]:
def find_claim(row, claim):
    try:
        return row.claims[0]['claim'] == claim
    except:
        return False

df[df.apply(lambda row: find_claim(row, 'le giec est transhumaniste'), axis=1)].id.values

array(['f17cb36a314156417616ef1ebb9232bbf203a1ecdc983c796a1a8ddba78393da'],
      dtype=object)

In [84]:
print(df['claims'].apply(lambda x: len(x)).unique())
print(df_cards['claims'].apply(lambda x: len(x)).unique())

[ 1  4  2  0  3  5 19  7  6 17  8  9]
[ 1  5  4  3  2  0  7  6 17  9 19  8]


In [86]:
df['claims'].iloc[1]

array([{'analysis': "L'allégation semble être fondée sur des données d'économies d'eau. Néanmoins, cela dépend de l'adoption généralisée de la technologie et des conditions de l'utilisation de l'eau dans les campings. Il serait prudent d'explorer si ces chiffres sont corroborés par des études ou des données réelles.", 'claim': "La facture s'allège de manière non négligeable avec cinquante pour cent d'économie d'eau atteignant parfois les six mille euros annuel.", 'context': "Dans une discussion sur une nouvelle innovation dans les campings qui vise à économiser de l'eau, il est mentionné que cette technologie permet de réduire la consommation d'eau et les coûts associés.", 'contradiction': 'N/A', 'disinformation_category': 'facts', 'disinformation_score': 'low', 'pro_anti': 'pro-écologie', 'quote': "La facture s'allège de manière non négligeable avec cinquante pour cent d'économie d'eau atteignant parfois les six mille euros annuel.", 'speaker': 'narrative'},
       {'analysis': "Cet é

In [89]:
df.iloc[1].id == df_cards.iloc[1].id

True

In [87]:
df_cards['claims'].iloc[1]

[{'analysis': "Cette allégation semble plausible et correspond à des études précédentes sur l'impact environnemental de l'industrie textile. Il est bien documenté que la production de coton est extrêmement consommatrice d'eau. Il semble donc que cela ne soit pas de la désinformation.",
  'claim': "Il faut environ 18 900 litres d'eau pour fabriquer un jean en coton.",
  'context': "Dans un discours abordant l'impact environnemental de l'habillement, il a été mentionné qu'un jean nécessite une quantité importante d'eau pour sa fabrication.",
  'contradiction': 'N/A',
  'disinformation_category': 'facts',
  'disinformation_score': 'very low',
  'pro_anti': 'pro-écologie',
  'quote': "Vous savez combien de litres d'eau il faut claude pour fabriquer un belot jean baudelot bawe le coton sur dix-huit cent neuf mille.",
  'speaker': 'consensus',
  'cards_label_pred': ''}]

In [90]:
df_cards.shape

(39131, 12)

In [91]:
df.shape

(39131, 12)

In [85]:
df['claims'].apply(lambda x: len(x)) == df_cards['claims'].apply(lambda x: len(x))

0         True
1        False
2        False
3         True
4        False
         ...  
39126    False
39127     True
39128     True
39129    False
39130    False
Name: claims, Length: 39131, dtype: bool

In [63]:
df[df.apply(lambda row: find_claim(row, 'des textes sur les énergies renouvelables'), axis=1)].id.values

array(['f632de53bfe36f76d420ff1295d2d9a2a18fc9cb8e560daf4d413246bd55f816'],
      dtype=object)

In [67]:
import pprint
pprint.pprint(df[df.id == 'f632de53bfe36f76d420ff1295d2d9a2a18fc9cb8e560daf4d413246bd55f816'].claims.values)

array([array([{'analysis': "L'allégation fait référence à une législation existante sur les énergies renouvelables. Il n'y a pas de désinformation ici, mais il est important de vérifier le contenu spécifique des textes pour comprendre leur impact et leur portée.", 'claim': 'des textes sur les énergies renouvelables', 'context': 'Le rapporteur mentionne les lignes adoptées concernant les énergies renouvelables tout en les intégrant dans un débat plus large sur différents textes de lois, y compris sur le nucléaire et le militaire.', 'contradiction': 'N/A', 'disinformation_category': 'facts', 'disinformation_score': 'very low', 'pro_anti': 'pro-écologie', 'quote': 'des textes sur les énergies renouvelables', 'speaker': 'consensus'},
              {'analysis': 'Cette affirmation aborde une réalité des politiques énergétiques actuelles sans indiquer de fausse information. Cela mérite cependant une vérification des détails des textes adoptés.', 'claim': 'adopter des textes sur le nucléaire',

In [72]:
pprint.pprint(df_cards[df_cards.id == 'f632de53bfe36f76d420ff1295d2d9a2a18fc9cb8e560daf4d413246bd55f816'].claims.values)

array(['[{\'analysis\': "L\'allégation fait référence à une législation existante sur les énergies renouvelables. Il n\'y a pas de désinformation ici, mais il est important de vérifier le contenu spécifique des textes pour comprendre leur impact et leur portée.", \'claim\': \'des textes sur les énergies renouvelables\', \'context\': \'Le rapporteur mentionne les lignes adoptées concernant les énergies renouvelables tout en les intégrant dans un débat plus large sur différents textes de lois, y compris sur le nucléaire et le militaire.\', \'contradiction\': \'N/A\', \'disinformation_category\': \'facts\', \'disinformation_score\': \'very low\', \'pro_anti\': \'pro-écologie\', \'quote\': \'des textes sur les énergies renouvelables\', \'speaker\': \'consensus\'}\n {\'analysis\': \'Cette affirmation aborde une réalité des politiques énergétiques actuelles sans indiquer de fausse information. Cela mérite cependant une vérification des détails des textes adoptés.\', \'claim\': \'adopter des 